In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
!pip install transformer_lens -q
!pip install --upgrade attrs -q
!pip uninstall jax jaxlib -y -q
!pip install jax jaxlib -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.9.2 requires numpy<1.26.0,>=1.18.5, but you have numpy 1.26.4 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.2.0 which is incompatible.


In [1]:
from transformer_lens import HookedTransformer
import torch

import json
import time

from src.utils import tuple_str_to_tuple
from src.neuron_texts import get_neuron_max_acts

In [14]:
# Load jsons from ../experiment_data/1_next_token_neurons
# filename = "2023-10-11_19-25-40_gpt2-xl"
# filename = "2023-10-11_19-07-38_gpt2-small"
# filename = "2023-10-11_19-19-59_gpt2-medium"
# filename = "2023-10-13_17-59-51_gpt2-large"
filename = "2024-02-13_04-54-06_pythia-160m"

# filename = "2024-02-13_04-33-18_pythia-1.4b"
# filename = "2024-02-15_07-29-20_pythia-410m"

train = False

with open(f'./experiment_data/1_next_token_neurons/{filename}.json') as f:
    neurons_data = json.load(f)

In [15]:
# Parameters
parameters = neurons_data['parameters']
model_name = parameters['model_name']
neurons_list = [tuple_str_to_tuple(x) for x in neurons_data['neurons'].keys()]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    # refactor_factored_attn_matrices=True,
    device=device,
)

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/375M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model pythia-160m into HookedTransformer


In [17]:
if train:
    from datasets import load_dataset
    dataset = load_dataset("NeelNanda/pile-10k", split="train")
    dataset_text_list = [x['text'] for x in dataset]
    print("Train loaded!")

else:
    import pickle

    with open('./text_list_dict_test.pkl', 'rb') as f:
        dataset_text_dict = pickle.load(f)

    dataset_text_list = [x['text'] for x in dataset_text_dict]
    print("Test loaded!")

Test loaded!


In [18]:
with torch.no_grad():
    neuron_max_acts = get_neuron_max_acts(
        model=model,
        dataset_text_list=dataset_text_list,
        neurons_list=neurons_list,
        batch_size=1,
        device=device,
    )

100%|██████████| 10000/10000 [20:51<00:00,  7.99it/s]


In [19]:
output = {
    'parameters': parameters,
    'neuron_max_acts': {str(key): value for key, value in neuron_max_acts.items()},
    'prior_filename': filename,
}

# Save json to ../experiment_data/2_max_activating_texts
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
train_str = "_train" if train else "_test_fr"
new_filename = f"{timestamp}_{model_name}{train_str}.json"
with open(f'./experiment_data/2_max_activating_texts/{new_filename}', 'w') as f:
    json.dump(output, f)

In [1]:
print(neuron_max_acts)

NameError: name 'neuron_max_acts' is not defined